In [1]:
import numpy as np
import tetrapolyscope as ps
import matplotlib.pyplot as plt

In [2]:
from chromalab.draw import IshiharaPlate, configurePolyscopeFor2D, configurePolyscopeForEvenOdd
from chromalab.observer import Observer, getSampledHyperCube
from chromalab.spectra import Illuminant
from chromalab.inks import get_metamer_buckets
from chromalab.maxbasis import MaxBasis
from chromalab.maxdisplaybasis import TetraDisplayGamut
from chromalab.cubemap import CubeMap

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths = np.arange(400, 700, 5)
d65 = Illuminant.get("D65")

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, verbose=True)
maxbasis  = MaxBasis(tetrachromat, verbose=True)


100%|██████████| 57/57 [00:03<00:00, 17.36it/s]


In [5]:
hypercube = getSampledHyperCube(0.019 , 4)

In [6]:
mdb = TetraDisplayGamut.loadTutenLabDisplay(tetrachromat, led_indices=[0, 3, 4, 5]) # R O C V
mdb.factor = 10000
intensities = mdb.primary_intensities.T * mdb.factor # columns are the ratios, also 10000 is the factor in order to get the get buckets function to work

In [7]:
all_lms_intensities = (intensities@hypercube.T).T # multiply all possible led combinations with the intensities
# buckets = get_metamer_buckets(all_lms_intensities, axis=2)

In [18]:
cubemap = CubeMap(all_lms_intensities, maxbasis, rgbs=np.zeros_like(all_lms_intensities), verbose=True)
cubemap.display_cubemap_RGBOCV(0.8, 0.4, 9, mdb)
# cubemap.display_(0.8, 0.4, 5) # lum val, sat val, grid

In [11]:
dist, (metamer_1, metamer_2) = buckets[0]

In [12]:
weights_4tup = mdb.convertActivationsToIntensities(np.array([np.array(metamer_1)/mdb.factor, np.array(metamer_2)/mdb.factor]).T)
# insert 0 for G and B channels
weights = np.insert(arr=weights_4tup, obj=[1,1], values=0, axis=1)

In [ ]:
print((mdb.primary_intensities.T@weights_4tup.T).T) # checking if they are metamers again after being transformed back to weights!!

In [14]:
ip = IshiharaPlate(weights[0], weights[1], 89, noise=50, image_size=912)

In [15]:
ip.generate_plate()

In [16]:
ip.export_even_odd_plate("metamer_pair_")

In [17]:
# while True:
#     ps.full_frame_tick()